In [136]:
import os
import json

import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R

from Motion.BVH import load, save
from Motion.Animation import Animation
from Motion.AnimationStructure import get_kinematic_chain
from Motion.Quaternions import Quaternions
from Motion.plot_script import plot_3d_motion

from body_comparison_realtime.utils import BODY38_idx2name, BODY38_name2idx

In [137]:
fname = "BP_Mixamo_New10_Scene_1_18_0.json"

with open(fname, "r") as fp:
    data = json.load(fp)

In [138]:
rotations = []
motion_data = data['motion_data']
for key in motion_data.keys():
    if key.endswith("RX") or key.endswith("RY") or key.endswith("RZ"):
        if motion_data[key]:
            rotations.append(motion_data[key])
rotations = np.array(rotations)
# plt.figure()
# plt.plot(rotations.T)
# plt.show()
rotations = rotations.T.reshape(rotations.shape[1], -1, 3)


translations = []
motion_data = data['motion_data']
for key in motion_data.keys():
    if key.endswith("TX") or key.endswith("TY") or key.endswith("TZ"):
        if motion_data[key]:
            translations.append(motion_data[key])
translations = np.array(translations)
# offsets = 12*np.ones((rotations.shape[0], 3))
offsets = translations[:, 0].reshape(-1, 3) /10
translations = translations.T.reshape(translations.shape[1], -1, 3)

orients = Quaternions(np.array([[1, 0, 0, 0] for _ in range(rotations.shape[0])]))

IndexError: tuple index out of range

In [130]:
new_anim = Animation(
    Quaternions.from_euler(rotations, order="xyz", world=False),
    np.zeros_like(rotations),
    orients,
    offsets,
    data["parents"]
)

In [131]:
save(filename="test.bvh", anim=new_anim, names=data["joints_names"], frametime=1/20)

In [132]:
# Initialize array to store 3D keypoints
keypoints = []

# Process each frame
for frame_idx in range(rotations.shape[0]):
    frame_keypoints = []  # Store keypoints for all joints in the current frame

    # Process each joint
    for joint_idx in range(rotations.shape[1]):
        euler_angles = rotations[frame_idx, joint_idx]
        translation = translations[frame_idx, joint_idx]

        # Convert Euler angles to a rotation matrix
        rotation_matrix = R.from_euler('xyz', euler_angles, degrees=True).as_matrix()

        # Apply the rotation and translation to the origin (0, 0, 0)
        keypoint = rotation_matrix @ np.array([0, 0, 0]) + translation

        frame_keypoints.append(keypoint)

    # Add all joint keypoints for the current frame
    keypoints.append(frame_keypoints)

# Convert to a numpy array for easy handling
keypoints = np.array(keypoints)  # Shape: (frames, joints, 3)

In [133]:
skeleton = get_kinematic_chain(data["parents"])
# plot_3d_motion("test_vis.mp4", skeleton, keypoints[:200], title="Test viz", fps=20)